In [1]:
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
from selenium import webdriver
from matplotlib import pyplot as plt
import re
import os,sys, shutil
import time
from datetime import date
try:
    import cPickle as pickle
except:
    import pickle
import pprint
from collections import deque
from shutil import copyfile
import random
import glob
# Import the required modules
import cv2, os
import numpy as np
from PIL import Image
from sklearn.cross_validation import KFold
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# Logistic Regression
from sklearn import datasets
from sklearn import metrics

import math
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

In [2]:
pkl_fl = open("linkedin_profiles.pickle","rb")
my_original_list=pickle.load(pkl_fl) # errors out here
pkl_fl.close()

In [3]:
directory = "Male"
    
if os.path.exists(directory):
    shutil.rmtree(directory)
    os.makedirs(directory)     
else:
    os.makedirs(directory) 

directory1 = "Female"

if os.path.exists(directory1):
    shutil.rmtree(directory1)
    os.makedirs(directory1)     
else:
    os.makedirs(directory1)     

directory2 = "Label_Images_Age"

if os.path.exists(directory2):
    shutil.rmtree(directory2)
    os.makedirs(directory2)     
else:
    os.makedirs(directory2)     
    
fileList = glob.glob("./Images/*.*")

for id,fp in enumerate(fileList):
    filename, file_extension = os.path.splitext(fp)
    uid = filename.split('/')[-1]
    #print fp
    for prof in my_original_list:
        if prof['User_ID'] == uid:
            prof_age = prof['age']
            
            if (0 <= prof_age <= 25):
                new_file_extension = 'Youth'
            elif (26 <= prof_age <= 35):
                new_file_extension = 'Adult'
            else:
                new_file_extension = 'Senior'
            
            copyfile(filename + ".jpg", './Label_Images_Age/'+ uid + '.' + str(id) + "." + new_file_extension +'.jpg')

In [3]:
# For face detection we will use the Haar Cascade provided by OpenCV.
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)

# For face recognition we will the the LBPH Face Recognizer 
recognizer = cv2.createLBPHFaceRecognizer()

In [4]:
def get_images_and_labels(path):
    # Append all the absolute image paths in a list image_paths
    
    image_paths = [os.path.join(path, f) for f in os.listdir(path)]
    # images will contains face images
    images = []
    # labels will contains the label that is assigned to the image
    labels = []
    #gender will contains 1 or 0 indecating male or female
    age =[]
    
    for image_path in image_paths:
        # Read the image and convert to grayscale
        try:
            image_pil = Image.open(image_path).convert('L')
            # Convert the image format into numpy array
            image = np.array(image_pil, 'uint8')
            # Get the label of the image
        except:
            pass
        
        nbr = int(os.path.split(image_path)[1].split(".")[1])
        age_current = os.path.split(image_path)[1].split(".")[2]
        print nbr
        
        # Detect the face in the image
        faces = faceCascade.detectMultiScale(image)
        # If face is detected, append the face to images and the label to labels
        try:
            for (x, y, w, h) in faces:

                ref_image = image[y: y + h, x: x + w]
                resized = cv2.resize(ref_image, (100, 100), interpolation = cv2.INTER_AREA)
                #edge_images = cv2.Canny(resized,100,200)
                
                images.append(np.array(resized))   #resized.reshape(1,10000)
                labels.append(nbr)

                if age_current == 'Youth':
                    age.append(0)
                elif age_current == 'Adult':
                    age.append(1)
                else:
                    age.append(2)
                
                #face_file_name = "faces/face_" + str(y) + ".jpg"
                #cv2.imwrite(face_file_name, sub_face)
                
                cv2.imshow("Adding faces to traning set...", resized)
                cv2.waitKey(1)
        except:
            pass
    # return the images list and labels list
    print "lables"
    print labels
    print "Age_current"
    print age
    
    return images, labels, age

In [5]:
images, labels, ageList = get_images_and_labels('Label_Images_Age')

cv2.destroyAllWindows()

2882
2780
2390
1533
775
1760
1973
2849
83
1414
1729
1566
1887
2098
2289
751
2367
2200
2318
891
771
2996
2268
2903
1767
737
370
1872
3323
2788
2292
1687
1374
2862
1644
3141
2501
2533
1501
920
2941
1314
1977
2529
1809
2212
1615
2543
2610
3354
1130
1951
3361
3255
2756
3261
479
784
2497
436
139
1377
1906
1976
1162
2489
1689
1235
1066
1293
2360
1806
2356
743
1960
1050
384
490
905
1630
2935
1148
2300
1226
448
1978
2355
2437
1829
678
53
1177
1833
1556
1003
1613
2109
2003
2403
2583
43
2755
1773
68
1388
1862
87
827
1696
2842
1021
1525
157
1965
2906
731
1180
1457
2444
1853
2532
587
510
1577
1805
250
2170
2165
330
382
2309
3171
2808
2905
1918
2515
938
3319
3200
1755
1528
2088
2778
2187
558
2042
2804
3115
3320
3074
132
1305
2819
3102
1811
2441
798
1619
813
3301
2495
1562
96
759
1750
2127
1929
646
213
2743
352
2410
2310
659
185
270
857
102
3131
1660
2010
900
99
2296
155
1458
1
1671
3355
2319
1417
591
1998
3032
1884
1028
2826
1399
471
1231
712
1923
2560
2736
374
689
877
407
764
128
2483
1527
807
230

In [6]:
#res_images = []
res_age = []

for age in ageList:
    res_age.append(np.array(age))
                   
res_age = np.array(res_age)

In [7]:
res_age.shape

(2909,)

## Basic model parameters as external flags.

In [8]:
# Basic model parameters as external flags.
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('max_steps', 2000, 'Number of steps to run trainer.')
flags.DEFINE_integer('hidden1', 1500, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('hidden2', 1000, 'Number of units in hidden layer 2.')
flags.DEFINE_integer('hidden3', 500, 'Number of units in hidden layer 3.')
flags.DEFINE_integer('batch_size', 100, 'Batch size.  '
                     'Must divide evenly into the dataset sizes.')
flags.DEFINE_string('train_dir', 'data', 'Directory to put the training data.')
flags.DEFINE_boolean('fake_data', False, 'If true, uses fake data '
                     'for unit testing.')


NUM_CLASSES = 3
IMAGE_SIZE = 100
#CHANNELS = 3
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE

In [9]:
n_nodes = [IMAGE_PIXELS, 1500, 1000, 500, NUM_CLASSES]
n_epochs = 10
#NUM_CLASSES = 3

In [ ]:
# def inference(images, hidden1_units, hidden2_units):
#     # Hidden 1
#     with tf.name_scope('hidden1'):
#         weights = tf.Variable(
#             tf.truncated_normal([IMAGE_PIXELS, hidden1_units],
#                                 stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),
#             name='weights')
#         biases = tf.Variable(tf.zeros([hidden1_units]),
#                              name='biases')
#         hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)
#     # Hidden 2
#     with tf.name_scope('hidden2'):
#         weights = tf.Variable(
#             tf.truncated_normal([hidden1_units, hidden2_units],
#                                 stddev=1.0 / math.sqrt(float(hidden1_units))),
#             name='weights')
#         biases = tf00,.Variable(tf.zeros([hidden2_units]),
#                              name='biases')
#         hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
#     # Linear
#     with tf.name_scope('softmax_linear'):
#         weights = tf.Variable(
#             tf.truncated_normal([hidden2_units, NUM_CLASSES],
#                                 stddev=1.0 / math.sqrt(float(hidden2_units))),
#             name='weights')
#         biases = tf.Variable(tf.zeros([NUM_CLASSES]),
#                              name='biases')
#         logits = tf.matmul(hidden2, weights) + biases
    
#     return logits

In [10]:
def neural_network_model(data):
    n_hidden_layers = 3
    # define the layers
    layers = [] 
    for i in range(n_hidden_layers + 1):
        layers.append( {'weights':tf.Variable(tf.random_normal([n_nodes[i], n_nodes[i+1]])), 
                        'biases':tf.Variable(tf.random_normal([n_nodes[i+1]]))} )
    
    # calculate the nodal values for each layer
    calcs = [data]
    for i in range(n_hidden_layers):
        calcs.append( tf.nn.relu(tf.matmul(calcs[i], layers[i]['weights']) + layers[i]['biases']) )

    #  return the last layer of nodes
    return tf.matmul(calcs[-1], layers[-1]['weights']) + layers[-1]['biases']

In [ ]:
# def cal_loss(logits, labels):
#     #labels = tf.to_int64(labels)
#     cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
#         logits, labels, name='xentropy')
#     loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
  
#     return loss

In [ ]:
# def training(loss, learning_rate):
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate)
#     global_step = tf.Variable(0, name='global_step', trainable=False)
#     train_op = optimizer.minimize(loss, global_step=global_step)
  
#     return train_op

In [11]:
def evaluation(logits, labels):
    correct = tf.nn.in_top_k(logits, labels, 1)
  
    return tf.reduce_sum(tf.cast(correct, tf.int32))

In [12]:
def placeholder_inputs():
    images_placeholder = tf.placeholder(tf.float32, [None,IMAGE_PIXELS])
    labels_placeholder = tf.placeholder(tf.float32, [None,NUM_CLASSES])
    
    return images_placeholder, labels_placeholder


def fill_feed_dict(images_feed,labels_feed, images_pl, labels_pl):
    feed_dict = {
      images_pl: images_feed,
      labels_pl: labels_feed,
    }
  
    return feed_dict


In [ ]:
# def do_eval(sess, eval_correct, images_placeholder, labels_placeholder, data_set):
#     # And run one epoch of eval.
#     true_count = 0  # Counts the number of correct predictions.
#     steps_per_epoch = 47 // FLAGS.batch_size
#     num_examples = steps_per_epoch * FLAGS.batch_size
#     for step in xrange(steps_per_epoch):
#         feed_dict = fill_feed_dict(train_images,train_labels,
#                                images_placeholder,
#                                labels_placeholder)
#         true_count += sess.run(eval_correct, feed_dict=feed_dict)
#     precision = true_count / num_examples
#     print('  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %
#         (num_examples, true_count, precision))

In [13]:
def dense_to_one_hot(labels_dense, num_classes):
    """Convert class labels from scalars to one-hot vectors."""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    
    return labels_one_hot

In [40]:
def run_training():
    # Tell TensorFlow that the model will be built into the default Graph.
    with tf.Graph().as_default():
        # Generate placeholders for the images and labels.
        images_placeholder, labels_placeholder = placeholder_inputs()
        
        
        #test_images_placeholder, test_labels_placeholder = placeholder_inputs(2)
        # Build a Graph that computes predictions from the inference model.
        
#         logits = inference(images_placeholder,
#                                  FLAGS.hidden1,
#                                  FLAGS.hidden2)
        
        logits = neural_network_model(images_placeholder)
        
        
        cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits,labels_placeholder) )
        
       
        #print cost
        optimizer = tf.train.AdamOptimizer(FLAGS.learning_rate).minimize(cost)

        with tf.Session() as sess:
            sess.run(tf.initialize_all_variables())

#             for epoch in range(n_epochs):
#                 epoch_loss = 0
#                 for _ in range( int(train_images.shape[0] / FLAGS.batch_size) ):
#                     feed_dict = fill_feed_dict(train_images,train_labels,
#                                            images_placeholder,
#                                            labels_placeholder)
                    
#                     #epoch_x, epoch_y = mnist.train.next_batch(batch_size)
#                     _, c = sess.run([optimizer, cost], feed_dict = feed_dict)
#                     epoch_loss += c
#                 print 'Epoch', epoch, 'completed out of', n_epochs, 'loss:', epoch_loss

#             correct = tf.equal(tf.argmax(logits,1), tf.argmax(labels_placeholder,1))
#             #print correct
#             accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
#             print 'Accuracy:', accuracy.eval({images_placeholder: test_images, labels_placeholder: test_labels})
    
#         # Add to the Graph the Ops for loss calculation.
#         loss = cal_loss(logits, labels_placeholder)

#         # Add to the Graph the Ops that calculate and apply gradients.
#         train_op = training(loss, FLAGS.learning_rate)

#         # Add the Op to compare the logits to the labels during evaluation.
#         eval_correct = evaluation(logits, labels_placeholder)

#         # Create a saver for writing training checkpoints.
#         saver = tf.train.Saver()

#         # Create a session for running Ops on the Graph.
#         sess = tf.Session()

#         # Run the Op to initialize the variables.
#         init = tf.initialize_all_variables()
#         sess.run(init)
        
        # And then after everything is built, start the training loop.
            
            subset_size = 100
            for step in xrange(10):
                start_time = time.time()
                total_loss = 0
                for i in range(int(train_images.shape[0] / subset_size) ):
                    
                    epoch_x = train_images[i * subset_size:][:subset_size]
                    epoch_y = train_labels[i * subset_size:][:subset_size]
                    
                    feed_dict = fill_feed_dict(epoch_x, epoch_y, images_placeholder, labels_placeholder)
                    
                    _, loss_value = sess.run([optimizer, cost],
                                           feed_dict=feed_dict)
                    
                    total_loss+=loss_value
                    
                duration = time.time() - start_time
                #if step % 10 == 0:
                    #Print status to stdout.
                print('Step %d: loss = %.2f (%.3f sec)' % (step, total_loss, duration))
                
                        
            correct = tf.equal(tf.argmax(logits,1), tf.argmax(labels_placeholder,1))
            #print correct
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print 'Accuracy:', accuracy.eval({images_placeholder: train_images, labels_placeholder: train_labels})

In [28]:
# Get the sets of images and labels for training, validation, and

train_images = np.array(images)
train_images = train_images.reshape(train_images.shape[0],IMAGE_PIXELS)

#label = res_gender
train_labels = dense_to_one_hot(res_age,NUM_CLASSES)

test_images = np.array(train_images[:100])
test_images = test_images.reshape(100,IMAGE_PIXELS)

#label = [0,1]
test_labels = np.array(train_labels[:100])

In [25]:
train_images.shape

(2909, 10000)

In [32]:
train_labels[:10]

array([[ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.]])

In [41]:
def main(_):
    run_training()

if __name__ == '__main__':
    tf.app.run()

Step 0: loss = 6751758900.00 (10.951 sec)
Step 1: loss = 2063777714.00 (11.392 sec)
Step 2: loss = 795529655.00 (17.197 sec)
Step 3: loss = 376589957.75 (17.097 sec)
Step 4: loss = 145948316.50 (17.378 sec)
Step 5: loss = 172188004.25 (17.281 sec)
Step 6: loss = 136210593.88 (17.515 sec)
Step 7: loss = 72079175.38 (18.295 sec)
Step 8: loss = 143230490.88 (17.691 sec)
Step 9: loss = 78932218.25 (17.673 sec)
Accuracy: 0.455483


SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.
